# License

In [2]:
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

def installDeps():
    !pip install numpy scipy
    !pip install resampy tensorflow six
    !pip install youtube_dl
    !pip install ipywidgets
    !pip install pydub
    !pip install tqdm
    !pip install ffmpeg-python
    !apt-get install ffmpeg -y
    #!wget https://storage.googleapis.com/audioset/vggish_model.ckpt
    #!wget https://storage.googleapis.com/audioset/vggish_pca_params.npz
#installDeps()
#!python vggish_train_demo.py --num_batches 50 --train_vggish=False --checkpoint './vggish_model.ckpt'

from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow as tf
import os
import vggish_input
import vggish_params
import vggish_slim
from pydub import AudioSegment
from audioModel import predict, train
from audioInput import getLaughTracks, getNoise

def trainAndSaveAndPredict(test_data, number_of_classes = 2, number_of_samples = 1, epochs = 5, getData = getLaughTracks, use_cache = True, log = True):
    def curriedGetSamples(shuf):
        return getData(number_of_samples = number_of_samples, shuf = shuf, use_cache = use_cache, log = log)
    model_name = 'model_%s_%s' % (number_of_samples, epochs)
    preds = train(curriedGetSamples, number_of_classes, model_name = model_name, epochs = epochs)
    

    return predict(model_name, number_of_classes, test_data)

def printResults(preds, expected = None): 
    with tf.Graph().as_default(), tf.Session() as sess:
        print(preds)
        print(sess.run(tf.argmax(input=preds, axis=1))) 
        print('expected results', expected)

def trainForNoise(number_of_samples=5, epochs=5):
    use_cache = False
    print('training on noise, sin, and constant waves')
    (features, labels) = getNoise(shuf=False, number_of_samples = 2)
    preds = trainAndSaveAndPredict(features, number_of_classes = 3, number_of_samples = number_of_samples, epochs = epochs, getData = getNoise)
    printResults(preds, [0, 0, 1, 1, 2, 2])
    
def trainForLaughter(number_of_samples=5, epochs=5):  
    use_cache = False
    #print('training on laughter and not laughter')
    (features, labels) = getLaughTracks(shuf=False, number_of_samples = 2, use_cache = use_cache, log=False)
    preds = trainAndSaveAndPredict(features, number_of_classes = 2, number_of_samples = number_of_samples, epochs = epochs, getData = getLaughTracks, use_cache = use_cache, log = False)
    #printResults(preds, [0, 0, 1, 1])


In [ ]:
trainForLaughter(number_of_samples=970, epochs=100)

INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
number of samples 950 reading 22 files ['samples/laughter/DZZORgAVFJw.wav', 'samples/laughter/EzS7y_3GhKA.wav', 'samples/laughter/R1hbmMfoT9c.wav']
number of samples 950 reading 23 files ['samples/notlaughter/hg6f7hh67aw.wav', 'samples/notlaughter/DUFBEamEF0Q.wav', 'samples/notlaughter/b0bUuVlesqw.wav']
Step 1: loss 0.756288
number of samples 950 reading 22 files ['samples/laughter/C80iIItZHFM.wav', 'samples/laughter/3IC76o_lhFw.wav', 'samples/laughter/IlMPU4AVU20.wav']
number of samples 950 reading 23 files ['samples/notlaughter/m7BgOYPNTlA.wav', 'samples/notlaughter/pM8kz_r2vqw.wav', 'samples/notlaughter/8OIH5gxUV1M.wav']
Step 3: loss 0.684908
number of samples 950 reading 22 files ['samples/laughter/iYVO5bUFww0.wav', 'samples/laughter/C80iIItZHFM.wav', 'samples/laughter/rHV09L1_t0g.wav']
number of samples 950 reading 23 files ['samples/notlaughter/8OIH5gxUV1M.wav', 'samples/notlaughter/Y3klZeiEkRY.wav', 'samples/notlaught

Step 26: loss 0.319671
number of samples 950 reading 22 files ['samples/laughter/Pz_DMUe4tXc.wav', 'samples/laughter/T_wNZhcw9x8.wav', 'samples/laughter/R1hbmMfoT9c.wav']
number of samples 950 reading 23 files ['samples/notlaughter/m7BgOYPNTlA.wav', 'samples/notlaughter/_zTpwNR5Bf4.wav', 'samples/notlaughter/hg6f7hh67aw.wav']
Step 27: loss 0.325721
number of samples 950 reading 22 files ['samples/laughter/16-lfVsYaxc.wav', 'samples/laughter/R1hbmMfoT9c.wav', 'samples/laughter/ySkafsRm9po.wav']
number of samples 950 reading 23 files ['samples/notlaughter/_zTpwNR5Bf4.wav', 'samples/notlaughter/-yUafzOXHPE.wav', 'samples/notlaughter/2tOwd3p7TsE.wav']
Step 28: loss 0.270524
number of samples 950 reading 22 files ['samples/laughter/iYVO5bUFww0.wav', 'samples/laughter/ySkafsRm9po.wav', 'samples/laughter/mbgrSdRs9bQ.wav']
number of samples 950 reading 23 files ['samples/notlaughter/TAqo-NZDuys.wav', 'samples/notlaughter/2tOwd3p7TsE.wav', 'samples/notlaughter/b0bUuVlesqw.wav']
Step 29: loss 0.

In [ ]:
#for i in range (0, 30):
#    number_of_samples = 1006 + (i * 1)
#    trainForLaughter(number_of_samples=number_of_samples, epochs=1)

INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
number of samples 1006 reading 22 files ['samples/laughter/DZZORgAVFJw.wav', 'samples/laughter/Pz_DMUe4tXc.wav', 'samples/laughter/3IC76o_lhFw.wav']
number of samples 1006 reading 23 files ['samples/notlaughter/2tOwd3p7TsE.wav', 'samples/notlaughter/pM8kz_r2vqw.wav', 'samples/notlaughter/zqAKTEfFOII.wav']
